<a href="https://colab.research.google.com/github/Nb4159/TreeTensorMNISTClassifier/blob/main/MNIST_Classification_using_Quantum_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing dependencies

In [1]:
! pip install tensorflow-quantum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.8/598.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 3.5 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12.1
    Uninstalling sympy-1.12.1:
      Successfully uninstalled sympy-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platfor

In [4]:
!pip install cirq

# Data

In [19]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Here I keep only digits 0-3
train_mask = (y_train < 4)
test_mask = (y_test < 4)
x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test, y_test = x_test[test_mask], y_test[test_mask]
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# Dimensionality reduction using PCA
pca = PCA(n_components=8)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

# Scale features
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train_pca)
x_test_scaled = scaler.transform(x_test_pca)

x_train_scaled



array([[0.6575419 , 0.51135594, 0.15022105, ..., 0.52434343, 0.3401515 ,
        0.46907204],
       [0.08702628, 0.66288   , 0.42997935, ..., 0.50262547, 0.5098418 ,
        0.42603576],
       [0.39441848, 0.7450557 , 0.47534683, ..., 0.39494413, 0.70676136,
        0.47028592],
       ...,
       [0.30617628, 0.81780624, 0.2842151 , ..., 0.19110218, 0.4387774 ,
        0.3972071 ],
       [0.09423772, 0.5708606 , 0.31920433, ..., 0.30449772, 0.46901459,
        0.37140912],
       [0.37251416, 0.4960603 , 0.61925966, ..., 0.18964401, 0.31799507,
        0.6993796 ]], dtype=float32)

# Amplitude Encoding
This is like making an embedding to feed data to the classifier in appropriate way. Amplitude Encoding encodes 2^n features into amplitude vector of n qubits.

# Tree Tensor Network[(TTN)](https://https://arxiv.org/abs/1803.11537)
TTNs are a type of tensor network that organizes tensors in a tree-like structure. They're used to represent quantum states efficiently.
Some of its features are:

**Hierarchical structure**: Information flows from the bottom (input) to the top (output).

**Logarithmic depth**: The number of layers scales logarithmically with the number of input qubits.

**Entanglement structure**: TTNs can capture short-range entanglement well but may struggle with long-range entanglement.



In [20]:
# Amplitude encoding
def amplitude_encode(features):
    qubits = cirq.GridQubit.rect(1, 8)
    circuit = cirq.Circuit()
    for i, feature in enumerate(features):
        circuit.append(cirq.ry(np.arccos(feature) * 2)(qubits[i]))
    return circuit

x_train_circuits = [amplitude_encode(x) for x in x_train_scaled]
x_test_circuits = [amplitude_encode(x) for x in x_test_scaled]

# Define the TTN model
def create_ttn_model():
    qubits = cirq.GridQubit.rect(1, 8)
    circuit = cirq.Circuit()

    # First layer
    for i in range(0, 8, 2):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+1]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i}'))(qubits[i]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+1}'))(qubits[i+1]))

    # Second layer
    for i in range(0, 8, 4):
        circuit.append(cirq.CNOT(qubits[i], qubits[i+2]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+8}'))(qubits[i]))
        circuit.append(cirq.ry(sympy.Symbol(f'θ_{i+9}'))(qubits[i+2]))

    # Third layer
    circuit.append(cirq.CNOT(qubits[0], qubits[4]))
    circuit.append(cirq.ry(sympy.Symbol('θ_16'))(qubits[0]))
    circuit.append(cirq.ry(sympy.Symbol('θ_17'))(qubits[4]))

    # Measurement
    circuit.append(cirq.measure(qubits[0], qubits[4], key='m'))

    readout_ops = [cirq.Z(qubits[0]), cirq.Z(qubits[4])]  # Readout operators

    pqc = tfq.layers.PQC(circuit, readout_ops)
    input_tensor = tf.keras.Input(shape=(), dtype=tf.string)
    output = pqc(input_tensor)
    output = tf.keras.layers.Dense(4, activation='softmax')(output)  # 4 output classes

    model = tf.keras.Model(inputs=[input_tensor], outputs=[output])

    return model, circuit

ttn_model,qc = create_ttn_model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
ttn_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])




<ipython-input-20-ee18192b752c>:6: RuntimeWarning: invalid value encountered in arccos
  circuit.append(cirq.ry(np.arccos(feature) * 2)(qubits[i]))
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [21]:
from tensorflow.keras.utils import plot_model
import os
plot_model(ttn_model, to_file='ttn_model.png', show_shapes=True, show_layer_names=True)

circuit_diagram = cirq.Circuit(qc)
print(circuit_diagram)


(0, 0): ───@───Ry(θ_0)───@───Ry(θ_8)────@───Ry(θ_16)───M('m')───
           │             │              │              │
(0, 1): ───X───Ry(θ_1)───┼──────────────┼──────────────┼────────
                         │              │              │
(0, 2): ───@───Ry(θ_2)───X───Ry(θ_9)────┼──────────────┼────────
           │                            │              │
(0, 3): ───X───Ry(θ_3)──────────────────┼──────────────┼────────
                                        │              │
(0, 4): ───@───Ry(θ_4)───@───Ry(θ_12)───X───Ry(θ_17)───M────────
           │             │
(0, 5): ───X───Ry(θ_5)───┼──────────────────────────────────────
                         │
(0, 6): ───@───Ry(θ_6)───X───Ry(θ_13)───────────────────────────
           │
(0, 7): ───X───Ry(θ_7)──────────────────────────────────────────


As we can see our neural network is in the form of a tree.

In [22]:
x_train_tfq = tfq.convert_to_tensor(x_train_circuits)
x_test_tfq = tfq.convert_to_tensor(x_test_circuits)

history = ttn_model.fit(
    x_train_tfq, y_train,
    batch_size=32,
    epochs=50,
    validation_split=0.2,
    verbose=1
)


test_loss, test_accuracy = ttn_model.evaluate(x_test_tfq, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


619/619 [==============================] - 50s 78ms/step - loss: nan - accuracy: 0.3111 - val_loss: nan - val_accuracy: 0.2426
Epoch 2/50
619/619 [==============================] - 47s 76ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 3/50
619/619 [==============================] - 46s 75ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 4/50
619/619 [==============================] - 45s 73ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 5/50
619/619 [==============================] - 47s 76ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 6/50
619/619 [==============================] - 52s 84ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 7/50
619/619 [==============================] - 46s 74ms/step - loss: nan - accuracy: 0.2384 - val_loss: nan - val_accuracy: 0.2426
Epoch 8/50
619/619 [=========================